### Install dependencies

In [1]:
# !pip install tiktoken --quiet

### Import libraries

In [2]:
import re
import json
import string
import tiktoken
from collections import Counter
from typing import List, Dict, Optional
import unicodedata
import threading
import time

import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional, Tuple, List
import numpy as np

from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm
import os

### Data Processor

In [3]:
import re
import unicodedata

class DataProcessor:
    """
    A utility class to clean and format raw text data for language model training.
    Attributes:
        endoftext_token (str): A special token appended to each processed text segment.
    """

    def __init__(self, endoftext_token: str = "<|endoftext|>"):
        self.endoftext_token = endoftext_token

    def clean_text(self, text: str) -> str:
        """
        Cleans input text by:
        - Normalizing Unicode characters to ASCII (NFKD normalization).
        - Removing all non-alphanumeric characters except spaces.
        - Collapsing multiple spaces into one.
        Args:
            text (str): The raw input text.
        Returns:
            str: The cleaned text.
        """
        # Normalize to ASCII using NFKD to remove accents/diacritics
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')
        # Remove all characters except alphanumeric and spaces
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        # Replace multiple spaces with a single space
        text = re.sub(r'\s+', ' ', text)
        return text.strip()

    def process_data(self, text: str) -> str:
        """
        Processes input text by cleaning it and appending the end-of-text token.
        Args:
            text (str): The raw input text.
        Returns:
            str: The cleaned and token-appended text.
        """
        cleaned_text = self.clean_text(text)
        processed_text = cleaned_text + f" {self.endoftext_token}"
        return processed_text


### Tiktoken Wrapper

In [4]:
from typing import List, Set
import tiktoken

class TiktokenTokenizer:
    """
    A wrapper around the TikToken tokenizer for encoding and decoding text using a specified model (e.g., GPT-2).
    Attributes:
        tokenizer: The TikToken encoding object for the specified model.
    """

    def __init__(self, model: str = "gpt2"):
        """
        Initializes the tokenizer with the encoding for the given model.
        Args:
            model (str): Name of the model to load encoding for. Default is "gpt2".
        """
        self.tokenizer = tiktoken.get_encoding(model)

    def encode(self, text: str, allowed_special: Set[str] = None) -> List[int]:
        """
        Encodes a text string into a list of token IDs, allowing specific special tokens.
        Args:
            text (str): The input string to tokenize.
            allowed_special (Set[str], optional): A set of allowed special tokens. 
                                                  Defaults to {"<|endoftext|>"} if not provided.
        Returns:
            List[int]: The list of token IDs representing the input string.
        """
        allowed_special = allowed_special or {"<|endoftext|>"}
        return self.tokenizer.encode(text, allowed_special=allowed_special)

    def decode(self, ids: List[int]) -> str:
        """
        Decodes a list of token IDs back into a string.
        Args:
            ids (List[int]): List of token IDs.
        Returns:
            str: The decoded string.
        """
        return self.tokenizer.decode(ids)

    def get_vocab_size(self) -> int:
        """
        Returns the size of the tokenizer's vocabulary
        Returns:
            int: The number of tokens in the vocabulary.
        """
        return self.tokenizer.n_vocab

### Load Text 

In [5]:
dataset_path = "./_dataset_/"
all_paths = os.listdir(dataset_path)
all_paths = sorted([os.path.join(dataset_path, path) for path in all_paths if path.endswith(".txt")])
print(all_paths)

raw_text = ""
for path in sorted(all_paths):
    with open(path, "r", encoding="utf-8") as f:
        raw_text += f.read().lower()

['./_dataset_/01 Harry Potter and the Sorcerers Stone.txt', './_dataset_/02 Harry Potter and the Chamber of Secrets.txt', './_dataset_/03 Harry Potter and the Prisoner of Azkaban.txt', './_dataset_/04 Harry Potter and the Goblet of Fire.txt', './_dataset_/05 Harry Potter and the Order of the Phoenix.txt', './_dataset_/06 Harry Potter and the Half-Blood Prince.txt', './_dataset_/07 Harry Potter and the Deathly Hallows.txt']


In [6]:
# Splits the raw_text corpus on new line character and joins it into a string
text = " ".join(raw_text.split("\n"))
len(text), text[:5000]    # Print the total character count of our text and first 5000 characters

(6285437,
 'mr. and mrs. dursley, of number four, privet drive, were proud to say that they were perfectly normal, thank you very much. they were the last people you’d expect to be involved in anything strange or mysterious, because they just didn’t hold with such nonsense.  mr. dursley was the director of a firm called grunnings, which made drills. he was a big, beefy man with hardly any neck, although he did have a very large mustache. mrs. dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors. the dursleys had a small son called dudley and in their opinion there was no finer boy anywhere.  the dursleys had everything they wanted, but they also had a secret, and their greatest fear was that somebody would discover it. they didn’t think they could bear it if anyone found out about the potters. mrs. potter was mrs. dursley’s sister, but they hadn’t met for

In [7]:
# Create an instance of data processor
processor = DataProcessor()
processed_text = processor.process_data(text)   # Clean the data

### Tokenize text corpus

In [8]:
tiktoken_tokenizer = TiktokenTokenizer()
tiktoken_encoded = tiktoken_tokenizer.encode(processed_text)
tiktoken_decoded = tiktoken_tokenizer.decode(tiktoken_encoded)

In [9]:
# Sanity check if tokenizer works fine
print("Processed text sample:", processed_text[:100])
print("Tiktoken encoded sample:", tiktoken_encoded[:10])
print("Tiktoken decoded sample:", tiktoken_decoded[:100])

Processed text sample: mr and mrs dursley of number four privet drive were proud to say that they were perfectly normal tha
Tiktoken encoded sample: [43395, 290, 285, 3808, 288, 1834, 1636, 286, 1271, 1440]
Tiktoken decoded sample: mr and mrs dursley of number four privet drive were proud to say that they were perfectly normal tha


In [10]:
# To obtain the tokenizer length and special_token token id
tiktoken_tokenizer.get_vocab_size(), tiktoken_tokenizer.encode("<|endoftext|>")

(50257, [50256])

### Config for model

In [11]:
class GPT2Config:
    """
    Configuration class for a GPT-2 model architecture.
    
    Stores all hyperparameters used to initialize and construct the GPT-2 model.
    """

    def __init__(
        self,
        vocab_size: int = 50257,             # Matches TikToken's GPT-2 tokenizer vocab size
        n_positions: int = 256,              # Maximum number of positional embeddings (i.e., sequence length)
        n_embd: int = 512,                   # Embedding and hidden layer size
        n_layer: int = 9,                    # Number of Transformer blocks
        n_head: int = 8,                     # Number of self-attention heads
        n_inner: int = None,                 # Feed-forward inner layer size (defaults to 4 * n_embd)
        activation_function: str = "gelu",   # Activation function used in MLP (e.g., gelu, relu)
        resid_pdrop: float = 0.1,            # Dropout rate for residual connections
        embd_pdrop: float = 0.1,             # Dropout rate for embeddings
        attn_pdrop: float = 0.1,             # Dropout rate within attention layers
        layer_norm_epsilon: float = 1e-5,    # Small epsilon value to avoid divide-by-zero in layer norm
        initializer_range: float = 0.02,     # Standard deviation for weight initialization
        eot_token_id: int = 50256            # Token ID representing <|endoftext|> in TikToken
    ):
        """
        Initializes the configuration with default or provided hyperparameters.
        
        Args:
            vocab_size (int): Size of the model vocabulary.
            n_positions (int): Maximum sequence length supported.
            n_embd (int): Dimension of embeddings and hidden layers.
            n_layer (int): Number of transformer layers.
            n_head (int): Number of attention heads.
            n_inner (int): Size of the feed-forward layer (defaults to 4 * n_embd).
            activation_function (str): Activation function in the MLP.
            resid_pdrop (float): Dropout probability for residual connections.
            embd_pdrop (float): Dropout probability for embeddings.
            attn_pdrop (float): Dropout probability for attention probabilities.
            layer_norm_epsilon (float): Epsilon value for layer normalization.
            initializer_range (float): Std for weight initialization.
            eot_token_id (int): Token ID used to signify end of text.
        """
        self.vocab_size = vocab_size
        self.n_positions = n_positions
        self.n_embd = n_embd
        self.n_layer = n_layer
        self.n_head = n_head
        self.n_inner = n_inner if n_inner is not None else 4 * n_embd
        self.activation_function = activation_function
        self.resid_pdrop = resid_pdrop
        self.embd_pdrop = embd_pdrop
        self.attn_pdrop = attn_pdrop
        self.layer_norm_epsilon = layer_norm_epsilon
        self.initializer_range = initializer_range
        self.eot_token_id = eot_token_id

### Model Modules

In [12]:
# Multi-Head Self-Attention module used in Transformer blocks
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, config: GPT2Config):
        super().__init__()
        assert config.n_embd % config.n_head == 0  # Ensure embedding size is divisible by num heads
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.head_dim = config.n_embd // config.n_head  # Dimension per head

        # Linear layer to project input into concatenated Q, K, V
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # Output projection layer
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)

        # Dropouts for attention probabilities and final projection
        self.attn_dropout = nn.Dropout(config.attn_pdrop)
        self.resid_dropout = nn.Dropout(config.resid_pdrop)

        # Causal mask (lower triangular) to prevent attending to future tokens
        self.register_buffer(
            "bias",
            torch.tril(torch.ones(config.n_positions, config.n_positions))
            .view(1, 1, config.n_positions, config.n_positions)
        )

    def forward(
        self,
        x: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        """
        Args:
            x (Tensor): Input tensor of shape (B, T, C)
            attention_mask (Optional[Tensor]): Optional attention mask

        Returns:
            Tuple[Tensor, Optional[Tensor]]: Output tensor and attention weights
        """
        B, T, C = x.size()

        # Compute Q, K, V from input
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)

        # Reshape Q, K, V for multi-head attention
        q = q.view(B, T, self.n_head, self.head_dim).transpose(1, 2)
        k = k.view(B, T, self.n_head, self.head_dim).transpose(1, 2)
        v = v.view(B, T, self.n_head, self.head_dim).transpose(1, 2)

        # Compute scaled dot-product attention scores
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(self.head_dim))
        att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))  # Apply causal mask

        if attention_mask is not None:
            # Apply additional attention mask if provided
            att = att.masked_fill(attention_mask[:, None, None, :] == 0, float('-inf'))

        # Normalize attention weights
        att = F.softmax(att, dim=-1)
        att = self.attn_dropout(att)

        # Compute context
        y = att @ v
        y = y.transpose(1, 2).contiguous().view(B, T, C)

        # Final projection and dropout
        y = self.resid_dropout(self.c_proj(y))
        return y, att


# Feed-forward network (MLP) used in each Transformer block
class MLP(nn.Module):
    def __init__(self, config: GPT2Config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, config.n_inner)  # First linear layer
        self.c_proj = nn.Linear(config.n_inner, config.n_embd)  # Second linear layer
        self.act = nn.GELU() if config.activation_function == "gelu" else nn.ReLU()  # Activation
        self.dropout = nn.Dropout(config.resid_pdrop)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of the MLP.

        Args:
            x (Tensor): Input tensor.

        Returns:
            Tensor: Output tensor after two linear layers and activation.
        """
        x = self.c_fc(x)
        x = self.act(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        return x


# Transformer block consisting of self-attention and MLP
class Block(nn.Module):
    def __init__(self, config: GPT2Config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd, eps=config.layer_norm_epsilon)
        self.attn = MultiHeadSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd, eps=config.layer_norm_epsilon)
        self.mlp = MLP(config)

    def forward(
        self,
        x: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Forward pass through one transformer block.

        Args:
            x (Tensor): Input tensor.
            attention_mask (Optional[Tensor]): Optional attention mask.

        Returns:
            Tuple[Tensor, Tensor]: Output tensor and attention weights.
        """
        attn_output, attn_weights = self.attn(self.ln_1(x), attention_mask)
        x = x + attn_output  # Residual connection
        mlp_output = self.mlp(self.ln_2(x))
        x = x + mlp_output  # Residual connection
        return x, attn_weights


# GPT-2 model architecture
class GPT2Model(nn.Module):
    def __init__(self, config: GPT2Config):
        super().__init__()
        self.config = config

        # Token and positional embeddings
        self.wte = nn.Embedding(config.vocab_size, config.n_embd)
        self.wpe = nn.Embedding(config.n_positions, config.n_embd)
        self.drop = nn.Dropout(config.embd_pdrop)

        # Transformer layers
        self.h = nn.ModuleList([Block(config) for _ in range(config.n_layer)])
        self.ln_f = nn.LayerNorm(config.n_embd, eps=config.layer_norm_epsilon)

        # Output projection head
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.lm_head.weight = self.wte.weight  # Tie input/output embeddings

        # Initialize model weights
        self.apply(self._init_weights)

    def _init_weights(self, module):
        # Initialize linear and embedding layers with normal distribution
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if isinstance(module, nn.Linear) and module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def forward(
        self,
        input_ids: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        """
        Forward pass through the GPT-2 model.

        Args:
            input_ids (Tensor): Input token IDs (B, T)
            attention_mask (Optional[Tensor]): Optional attention mask.
            labels (Optional[Tensor]): Labels for computing loss.

        Returns:
            Tuple[Tensor, Optional[Tensor]]: Logits and optional loss.
        """
        B, T = input_ids.size()
        device = input_ids.device

        # Generate position IDs
        position_ids = torch.arange(0, T, dtype=torch.long, device=device).unsqueeze(0).expand(B, T)

        # Embed tokens and positions
        inputs_embeds = self.wte(input_ids)
        position_embeds = self.wpe(position_ids)
        x = self.drop(inputs_embeds + position_embeds)

        # Pass through transformer layers
        attn_weights = []
        for block in self.h:
            x, attn = block(x, attention_mask)
            attn_weights.append(attn)

        x = self.ln_f(x)

        # Final projection to vocab size
        logits = self.lm_head(x)

        # Compute loss if labels are provided
        loss = None
        if labels is not None:
            loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)),
                labels.view(-1),
                ignore_index=-100
            )
        return logits, loss

    def generate(
        self,
        input_ids: torch.Tensor,
        max_new_tokens: int = 50,
    ) -> torch.Tensor:
        """
        Greedy token-by-token generation.

        Args:
            input_ids (Tensor): Initial input tensor.
            max_new_tokens (int): Number of tokens to generate.

        Returns:
            Tensor: Generated token IDs.
        """
        self.eval()
        batch_size = input_ids.size(0)
        generated = input_ids

        with torch.no_grad():
            for _ in range(max_new_tokens):
                logits, _ = self(generated.to('cuda'))
                next_token = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
                generated = torch.cat([generated, next_token], dim=1)
        return generated


### Create model 

In [13]:
config = GPT2Config()
model = GPT2Model(config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2Model(
  (wte): Embedding(50257, 512)
  (wpe): Embedding(256, 512)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-8): 9 x Block(
      (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (attn): MultiHeadSelfAttention(
        (c_attn): Linear(in_features=512, out_features=1536, bias=True)
        (c_proj): Linear(in_features=512, out_features=512, bias=True)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (mlp): MLP(
        (c_fc): Linear(in_features=512, out_features=2048, bias=True)
        (c_proj): Linear(in_features=2048, out_features=512, bias=True)
        (act): GELU(approximate='none')
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (lm_head): Linear(in_features=512, out_features=50257, bias=False)
)

#### Test model

In [18]:
prompt = "Professor McGonagall sniffed angrily".lower()
input_ids = torch.tensor([tiktoken_tokenizer.encode(prompt)], dtype=torch.long).to(device)

generated_ids = model.generate(
        input_ids,
        max_new_tokens=50,
    )

generated_text = tiktoken_tokenizer.decode(generated_ids[0].cpu().numpy().tolist())


print("Prompt:", prompt)
print("Generated text:", generated_text)

assert torch.allclose(model.wte.weight, model.lm_head.weight), "Weights are not tied!"
print("Weight tying verified.")

total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,}")

Prompt: professor mcgonagall sniffed angrily
Generated text: professor mcgonagall sniffed angrily antigen head head head head head head head head head head head head Show Show Show speech speech speech speech speech speech speech speech speech speech speech speech speech speech speech speech speech speech speech speech My My My history history history history history history history history history history history
Weight tying verified.
Total parameters: 54,235,136


### Training hyperparameters

In [14]:
def get_hyperparameters(config: GPT2Config):
    return {
        "learning_rate": 1e-4,  # Typical for GPT-2
        "batch_size": 4,        # Small batch size for demonstration
        "num_epochs": 300,       # Maximum epochs
        "max_seq_length": 64,  # Shorter sequences for faster training
        "patience": 5,          # Early stopping patience
        "train_split" : 0.9,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "model_config": config  # Reuse GPT2Config
    }

In [15]:
hparams = get_hyperparameters(config)

### Create dataloader

In [16]:
class TextDataset(Dataset):
    def __init__(self, tokens: List[int], max_seq_length: int):
        self.tokens = tokens
        self.max_seq_length = max_seq_length

    def __len__(self):
        return len(self.tokens) // self.max_seq_length

    def __getitem__(self, idx):
        start = idx * self.max_seq_length
        end = start + self.max_seq_length
        input_ids = torch.tensor(self.tokens[start:end], dtype=torch.long)
        target_ids = torch.tensor(self.tokens[start+1:end+1], dtype=torch.long)
        return input_ids, target_ids

In [17]:
def get_data_loaders(text: str, hparams: dict, tokenizer: TiktokenTokenizer):
    tokens = tokenizer.encode(text)

    split_idx = int(len(tokens) * hparams["train_split"])
    train_tokens = tokens[:split_idx]
    val_tokens = tokens[split_idx:]

    train_dataset = TextDataset(train_tokens, hparams["max_seq_length"])
    val_dataset = TextDataset(val_tokens, hparams["max_seq_length"])

    train_loader = DataLoader(
        train_dataset,
        batch_size=hparams["batch_size"],
        shuffle=False,
        num_workers=8
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=hparams["batch_size"],
        shuffle=False,
        num_workers=8
    )

    return train_loader, val_loader


In [18]:
train_loader, val_loader = get_data_loaders(processed_text, hparams, tiktoken_tokenizer)

### Train model

In [19]:
def train_model(model: GPT2Model, train_loader: DataLoader, val_loader: DataLoader, hparams: dict, tokenizer: TiktokenTokenizer):
    model.to(hparams["device"])
    optimizer = AdamW(model.parameters(), lr=hparams["learning_rate"])
    scheduler = CosineAnnealingLR(optimizer, T_max=500)

    best_train_loss = float('inf')
    patience_counter = 0
    best_model_path = "best_gpt2_model.pt"

    for epoch in range(hparams["num_epochs"]):
        start_time = time.time()

        # Training
        model.train()
        train_loss = 0.0
        train_steps = 0
        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{hparams['num_epochs']} Training")
        for input_ids, target_ids in train_pbar:
            input_ids = input_ids.to(hparams["device"])
            target_ids = target_ids.to(hparams["device"])

            optimizer.zero_grad()
            logits, loss = model(input_ids, labels=target_ids)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_steps += 1
            train_pbar.set_postfix({"train_loss": train_loss / train_steps})

        scheduler.step()
        avg_train_loss = train_loss / train_steps

        epoch_time = time.time() - start_time

        # Print epoch results
        print(f"""Epoch {epoch+1}/{hparams['num_epochs']} | Train Loss: {avg_train_loss:.4f} | Time: {epoch_time:.2f}s""") # | Validation Loss: {avg_val_loss:.4f}""")

        # Early stopping and model saving
        if avg_train_loss < best_train_loss:
            best_val_loss = avg_train_loss
            patience_counter = 0
            torch.save(model.state_dict(), best_model_path)
            print(f"Saved best model to {best_model_path}")
        else:
            patience_counter += 1
            print(f"Patience counter: {patience_counter}/{hparams['patience']}")
            if patience_counter >= hparams["patience"]:
                print("Early stopping triggered.")
                break
        print()
    # Load best model
    model = GPT2Model(config).to(hparams["device"])
    model.load_state_dict(torch.load(best_model_path))
    print(f"Loaded best model from {best_model_path}")
    return model, best_val_loss

In [22]:
model = train_model(model, train_loader, val_loader, hparams, tiktoken_tokenizer)

Epoch 1/300 Training: 100%|██████████| 70/70 [00:25<00:00,  2.74it/s, train_loss=8.42]


Epoch 1/300 | Train Loss: 8.4178 | Time: 25.58s
Saved best model to best_gpt2_model.pt



Epoch 2/300 Training: 100%|██████████| 70/70 [00:25<00:00,  2.75it/s, train_loss=6.79]


Epoch 2/300 | Train Loss: 6.7938 | Time: 25.45s
Saved best model to best_gpt2_model.pt



Epoch 3/300 Training: 100%|██████████| 70/70 [00:25<00:00,  2.75it/s, train_loss=6.34]


...

Epoch 298/300 | Train Loss: 0.2125 | Time: 25.52s
Saved best model to best_gpt2_model.pt



Epoch 299/300 Training: 100%|██████████| 70/70 [00:25<00:00,  2.74it/s, train_loss=0.211]


Epoch 299/300 | Train Loss: 0.2111 | Time: 25.51s
Saved best model to best_gpt2_model.pt



Epoch 300/300 Training: 100%|██████████| 70/70 [00:25<00:00,  2.74it/s, train_loss=0.211]


Epoch 300/300 | Train Loss: 0.2107 | Time: 25.52s
Saved best model to best_gpt2_model.pt

Loaded best model from best_gpt2_model.pt


In [27]:
model = GPT2Model(config).to(hparams['device'])
model.load_state_dict(torch.load("best_gpt2_model_Final_v1.pt"))

<All keys matched successfully>

In [35]:
import random

# Select 10 random indices from the dataset
random_ids = random.sample(range(len(train_loader)), 5)

for idx in random_ids:
    print("*"*50, f"SAMPLE {idx}", "*"*50)
    input_ids, target_ids = train_loader.dataset[idx]  # Use dataset indexing directly
    input_text = tiktoken_tokenizer.decode(input_ids.tolist())
    target_text = tiktoken_tokenizer.decode(target_ids.tolist())

    generated_ids = model.generate(
        input_ids.unsqueeze(0).to(hparams['device']),  # Add batch dimension
        max_new_tokens=20,
    )
    generated_text = tiktoken_tokenizer.decode(generated_ids[0].cpu().tolist())

    print("Prompt: ", input_text, "\n")
    print("Target text: ", target_text, "\n")
    print("Generated text: ", generated_text, "\n")

************************************************** SAMPLE 2473 **************************************************
Prompt:   book some of the books the ministrys confiscated dads told me there was one that burned your eyes out and everyone who read sonnets of a sorcerer spoke in limericks for the rest of their lives and some old witch in bath had a book that you could never stop reading you just had to wander around with your nose 

Target text:   some of the books the ministrys confiscated dads told me there was one that burned your eyes out and everyone who read sonnets of a sorcerer spoke in limericks for the rest of their lives and some old witch in bath had a book that you could never stop reading you just had to wander around with your nose in 

Generated text:   book some of the books the ministrys confiscated dads told me there was one that burned your eyes out and everyone who read sonnets of a sorcerer spoke in limericks for the rest of their lives and some old witch in bath h

### Check model perplexity

In [41]:
def evaluate_perplexity(model, data_loader, device):
    model.eval()
    total_loss = 0.0
    total_tokens = 0

    with torch.no_grad():
        for input_ids, target_ids in data_loader:
            input_ids = input_ids.to(device)
            target_ids = target_ids.to(device)

            logits, loss = model(input_ids, labels=target_ids)
            num_tokens = target_ids.numel()

            total_loss += loss.item() * num_tokens
            total_tokens += num_tokens

    avg_loss = total_loss / total_tokens
    perplexity = math.exp(avg_loss)
    return perplexity

In [46]:
perplexity = evaluate_perplexity(model, train_loader, hparams["device"])
print(f"Perplexity on training data: {perplexity:.2f}")

Perplexity on training data: 1.06
